In [1]:
# %pip install ftfy regex tqdm
# %pip install colorama

In [2]:
import jittor as jt
from PIL import Image
import jclip as clip
import os
from tqdm import tqdm
import argparse
from sklearn.linear_model import LogisticRegression
import numpy as np
# from colorama import Fore, Back, Style, init
# init()

jt.flags.use_cuda = 1
print("包导入成功")

[i 0614 18:11:35.367531 16 log.cc:351] Load log_sync: 1
[i 0614 18:11:35.429343 16 compiler.py:956] Jittor(1.3.8.5) src: /root/miniconda3/envs/pyt/lib/python3.10/site-packages/jittor
[i 0614 18:11:35.432729 16 compiler.py:957] g++ at /usr/bin/g++(9.4.0)
[i 0614 18:11:35.433511 16 compiler.py:958] cache_path: /root/.cache/jittor/jt1.3.8/g++9.4.0/py3.10.11/Linux-5.4.0-16xf1/IntelRXeonRPlax40/default
[i 0614 18:11:35.437135 16 __init__.py:411] Found /usr/local/cuda/bin/nvcc(11.7.99) at /usr/local/cuda/bin/nvcc.
[i 0614 18:11:35.440178 16 __init__.py:411] Found addr2line(2.34) at /usr/bin/addr2line.


[i 0614 18:11:35.553101 16 compiler.py:1011] cuda key:cu11.7.99_sm_75
[i 0614 18:11:35.897398 16 __init__.py:227] Total mem: 30.60GB, using 10 procs for compiling.
[i 0614 18:11:36.043001 16 jit_compiler.cc:28] Load cc_path: /usr/bin/g++
[i 0614 18:11:36.269060 16 init.cc:62] Found cuda archs: [75,]
[i 0614 18:11:39.370891 16 cuda_flags.cc:49] CUDA enabled.


包导入成功


In [3]:
parser = argparse.ArgumentParser()
parser.add_argument('--split', type=str, default='A')

# args = parser.parse_args()
args, unknown = parser.parse_known_args()
model, preprocess = clip.load("ViT-B-32.pkl")
classes = open('../data/classname.txt').read().splitlines()#这是一个包含所有类的列表

## 类别文本编码,变成一句话,然后别转为向量
### 提示词可以尝试优化
text_features的shape为[374,512,]

In [4]:
# encode这块后面可以强化下
new_classes = []
for c in classes:
    c = c.split(' ')[0]
    if c.startswith('Animal'):
        c = c[7:]
    if c.startswith('Thu-dog'):
        c = c[8:]
    if c.startswith('Caltech-101'):
        c = c[12:]
    if c.startswith('Food-101'):
        c = c[9:]
    c = 'a photo of ' + c
    new_classes.append(c)
    text = clip.tokenize(new_classes)
text_features = model.encode_text(text)
text_features /= text_features.norm(dim=-1, keepdim=True)
# print(Back.YELLOW +"标签处理成功")

## 图片加载并处理

In [5]:
# training data loading
imgs_dir = '../data'
train_labels = open('../data/train.txt').read().splitlines()
train_imgs = [l.split(' ')[0] for l in train_labels]#对应的图片path
train_labels = [jt.float32([int(l.split(' ')[1])]) for l in train_labels]#对应的种类序号

[i 0614 18:11:48.567942 16 cuda_device_allocator.cc:30] 
=== display_memory_info ===
 total_cpu_ram:  30.6GB total_device_ram: 14.58GB
 hold_vars: 49212 lived_vars: 99467 lived_ops: 100000
 name: sfrl is_device: 1 used: 11.37GB(93%) unused: 872.7MB(6.97%) total: 12.23GB
 name: sfrl is_device: 1 used: 579.8MB(91.6%) unused: 53.24MB(8.41%) total:   633MB
 name: sfrl is_device: 0 used: 579.8MB(91.6%) unused: 53.24MB(8.41%) total:   633MB
 name: sfrl is_device: 0 used: 23.91MB(99.6%) unused:    88KB(0.358%) total:    24MB
 name: sfrl is_device: 0 used:     0 B(-nan%) unused:     0 B(-nan%) total:     0 B
 name: temp is_device: 0 used:     0 B(-nan%) unused:     0 B(-nan%) total:     0 B
 name: temp is_device: 1 used:     0 B(-nan%) unused:     0 B(-nan%) total:     0 B
 cpu&gpu: 13.49GB gpu: 12.84GB cpu:   657MB
 free: cpu(20.62GB) gpu(15.38MB)
 swap: total(    0 B) last(    0 B)

[i 0614 18:11:49.196216 16 cuda_device_allocator.cc:30] 
=== display_memory_info ===
 total_cpu_ram:  30.6GB t

RuntimeError: [38;5;1m[f 0614 18:11:49.197200 16 executor.cc:686] 
Execute fused operator(1383/99182) failed. 
[JIT Source]: /root/.cache/jittor/jt1.3.8/g++9.4.0/py3.10.11/Linux-5.4.0-16xf1/IntelRXeonRPlax40/default/cu11.7.99_sm_75/jit/code__IN_SIZE_1__in0_dim_4__in0_type_float32__OUT_SIZE_1__out0_dim_4__out0_type_float32__H___hash_9ea216850b079559_op.cc 
[OP TYPE]: code 
[Input]: float32[374,8,77,77,], 
[Output]: float32[374,8,77,77,], 
[Async Backtrace]: not found, please set env JT_SYNC=1, trace_py_var=3 
[Reason]: [38;5;1m[f 0614 18:11:49.197092 16 cuda_device_allocator.cc:31] Unable to alloc cuda device memory for size 71303168[m[m
**********
Async error was detected. To locate the async backtrace and get better error report, please rerun your code with two enviroment variables set:
>>> export JT_SYNC=1
>>> export trace_py_var=3


In [ ]:
# 每个类挑四张图，根据train_labels中的label来挑选
#挑选每种的前四张,生成两个对应的列表,分别存储path和类别信息
cnt = {}
new_train_imgs = []
new_train_labels = []
for i in range(len(train_imgs)):
    label = int(train_labels[i].numpy())
    if label not in cnt:
        cnt[label] = 0
    if cnt[label] < 4:
        new_train_imgs.append(train_imgs[i])
        new_train_labels.append(train_labels[i])
        cnt[label] += 1

In [ ]:
# calculate image features of training data
train_features = []
print('Training data processing:')
with jt.no_grad():
    for img in tqdm(new_train_imgs):
        img = os.path.join(imgs_dir, img)
        image = Image.open(img)
        image = preprocess(image).unsqueeze(0)
        image_features = model.encode_image(image)
        print("能成功运行?","image_features的shape是",image_features.shape)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        train_features.append(image_features)

train_features = jt.cat(train_features).numpy()
train_labels = jt.cat(new_train_labels).numpy()

In [ ]:
train_features[0]